## Reaction-diffusion model

The aim of this notebook is to provide insight with devito into so-called reaction-diffusion models. These are problems which, intuitively, involve some reaction that occurs, and results in some form of diffusion.

This may sound initially vague, but has a variety of application that illustrate its meaning. The first of these is cell multiplication -- the cell population will diffuse outwards, following the reaction of duplication which individual cells undergo.

This problem is an answer to Lorena Barba's reaction-diffusion notebook which can be found <a href="https://github.com/numerical-mooc/numerical-mooc/blob/master/lessons/04_spreadout/04_06_Reaction_Diffusion.ipynb">here</a>. Like our CFD examples, we start by setting up the problem, and will then create a quick and easy Devito solution to it.

We start by introducing the problem.

#### The problem ####

<ul>
    <li> Discretize the reaction-diffusion equations using forward-time/central-space and assume that $$ \Delta x = \Delta y = \delta $$
    <li> For our timestep, set $$ \Delta t = \frac{9}{40} \frac{\delta ^ 2}{max(D_u, D_v)} $$
    <li> Use zero Neumann boundary conditions on all sides of the domain.
</ul>

The initial conditions and constants we are given are listed in the cell below, and correspond to the domain:

<ul>
    <li> A grid with dimensions 192 $ \times $ 192 points
    <li> Domain is the 5 m $ \times $ 5 m
    <li> Final time is 8000 s
</ul>

In [9]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [10]:
n = 192

Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 

dh = 5/(n-1)

T = 8000

dt = .9 * dh**2 / (4*max(Du,Dv))

nt = int(T/dt)

#### Initial Conditions ####

Also given to us in the problem is the initial conditions which contains two numpy arrays which hold the initial values for `U` and `V`, respectively.

In [ ]:
uvinitial = numpy.load('./data/uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

In [ ]:
fig = pyplot.figure(figsize=(8,5))
pyplot.subplot(121)
pyplot.imshow(U, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);
pyplot.subplot(122)
pyplot.imshow(V, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);

#### Devito Implementation ####

